In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install datasets ipywidgets chromadb -q


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [14]:
!pip install sentence_transformers -q


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [11]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction


In [ ]:
# Set the Hugging Face token
os.environ["HF_TOKEN"] = ""

In [6]:
# Load the model and tokenizer
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=os.getenv("HF_TOKEN"))
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.getenv("HF_TOKEN"))


/usr/local/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

/usr/local/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
# Assign the eos_token as the pad_token (if needed)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
# Load the dataset
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")["train"]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [12]:
# Initialize Chroma DB
client = chromadb.Client(Settings(persist_directory="./chroma_db", anonymized_telemetry=False))


In [15]:
# Create an embedding function for Chroma DB
embedding_fn = SentenceTransformerEmbeddingFunction("all-MiniLM-L6-v2")


E0000 00:00:1724354290.031630      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
E0822 19:18:10.064823469      13 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2024-08-22T19:18:10.064800245+00:00"}


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# Add documents to Chroma DB
collection = client.create_collection("medquad", embedding_function=embedding_fn)


In [17]:
# Prepare data for Chroma DB
data = [{"id": str(i), "document": dataset[i]["Answer"]} for i in range(len(dataset))]

# Add data to the Chroma collection
collection.add(ids=[item["id"] for item in data],
               documents=[item["document"] for item in data])

In [18]:
def retrieve_documents_chroma(question, collection, top_n=5):
    # Retrieve top_n most similar documents from Chroma DB
    results = collection.query(query_texts=[question], n_results=top_n)
    
    # Extract documents from the results
    retrieved_docs = results['documents'][0]
    
    return " ".join(retrieved_docs)


In [19]:
def rag_pipeline_chroma(question, collection, model, tokenizer):
    # Retrieve relevant documents using Chroma DB
    docs = retrieve_documents_chroma(question, collection)

    # Concatenate the question with the retrieved documents
    combined_input = question + " " + docs

    # Tokenize the combined input
    inputs = tokenizer(combined_input, return_tensors="pt", padding=True, truncation=True, max_length=256)

    # Generate an answer using the model
    outputs = model.generate(
        inputs["input_ids"], 
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50
    )

    # Decode and return the generated answer
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [20]:
# Example usage
question = "Who is at risk for Lymphocytic Choriomeningitis (LCM)?"
answer = rag_pipeline_chroma(question, collection, model, tokenizer)
print("Answer:", answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: Who is at risk for Lymphocytic Choriomeningitis (LCM)? Langerhans cell histiocytosis (LCH) is a disorder that primarily affects children, but is also found in adults of all ages. People with LCH produce too many Langerhans cells or histiocytes, a form of white blood cell found in healthy people that is supposed to protect the body from infection. In people with LCH, these cells multiply excessively and build up in certain areas of the body, causing tumors called granulomas to form. The symptoms vary among affected individuals, and the cause of LCH is unknown. In most cases, this condition is not life-threatening. Some people do experience life-long problems associated with LCH. Multicentric Castleman disease (MCD) is a rare condition that affects the lymph nodes and related tissues. It is a form of Castleman disease that is "systemic" and affects multiple sets of lymph nodes and other tissues throughout the body (as opposed to unicentric Castleman disease which has more "locali

In [21]:
# Example usage
question = "How to diagnose Parasites - Cysticercosis ?"
answer = rag_pipeline_chroma(question, collection, model, tokenizer)
print("Answer:", answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: How to diagnose Parasites - Cysticercosis? If you think that you may have cysticercosis, please see your health care provider. Your health care provider will ask you about your symptoms, where you have travelled, and what kinds of foods you eat. The diagnosis of neurocysticercosis usually requires MRI or CT brain scans. Blood tests may be useful to help diagnose an infection, but they may not always be positive in light infections.
  
If you have been diagnosed with cysticercosis, you and your family members should be tested for intestinal tapeworm infection. See the taeniasis section for more information on intestinal tapeworm infections.  
  
More on: Taeniasis
  
More on: Resources for Health Professionals: Diagnosis Cysticercosis is an infection caused by the larvae of the parasite Taenia solium.  This infection occurs after a person swallows tapeworm eggs. The larvae get into tissues such as muscle and brain, and form cysts there (these are called cysticerci). When cysts a

In [22]:
# Example usage
question = "How to prevent Marburg hemorrhagic fever (Marburg HF) ?"
answer = rag_pipeline_chroma(question, collection, model, tokenizer)
print("Answer:", answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: How to prevent Marburg hemorrhagic fever (Marburg HF)? There is no specific treatment for Marburg hemorrhagic fever. Supportive hospital therapy should be utilized, which includes balancing the patient's fluids and electrolytes, maintaining oxygen status and blood pressure, replacing lost blood and clotting factors, and treatment for any complicating infections. 
 
Experimental treatments are validated in non-human primates models, but have never been tried in humans. Preventive measures against Marburg virus infection are not well defined, as transmission from wildlife to humans remains an area of ongoing research. However, avoiding fruit bats, and sick non-human primates in central Africa, is one way to protect against infection. 
 
Measures for prevention of secondary, or person-to-person, transmission are similar to those used for other hemorrhagic fevers. If a patient is either suspected or confirmed to have Marburg hemorrhagic fever, barrier nursing techniques should be u